## Load Libraries and Packages

In [1]:
pwd

u'/Users/jfdarre/Documents/NYCDS/Project4/KaggleProject'

In [2]:
import datetime
import numpy as np
from numpy import double
import pandas as pd

## Load Data

In [3]:
print("Loading data start...")
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
store = pd.read_csv("../input/store.csv")
states = pd.read_csv("../input/store_states.csv")
print("Complete!   : )")

Loading data start...
Complete!   : )


/Users/jfdarre/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


## Process Data:
#### 1: Impute open = 1 for missing open test dasta
#### 2: Separate date column into year, month, day
#### 3: Convert date column to type 'date' and extract:
* day_of_year
* week_of_year
* quarter
* month start
* month end
* quarter start
* quarter end

#### 4: Drop date and standardize StateHoliday
#### 5: Create promotion one feature
#### 6: Merge data
#### 7: Dumify categorical variables

In [4]:
# 1: Impute Open = 1 for store 622 in test data
test['Open'].fillna(1, inplace=True)
print 'Step 1 Complete.'

Step 1 Complete.


In [5]:
# 2: Separate date columns
train['year'] = train.Date.apply(lambda x: x.split('-')[0])
train['year'] = train['year'].astype(int)
train['month'] = train.Date.apply(lambda x: x.split('-')[1])
train['month'] = train['month'].astype(int)
train['day'] = train.Date.apply(lambda x: x.split('-')[2])
train['day'] = train['day'].astype(int)
print 'Step 2 Complete.'

Step 2 Complete.


In [6]:
# 2: Separate date columns
test['year'] = test.Date.apply(lambda x: x.split('-')[0])
test['year'] = test['year'].astype(int)
test['month'] = test.Date.apply(lambda x: x.split('-')[1])
test['month'] = test['month'].astype(int)
test['day'] = test.Date.apply(lambda x: x.split('-')[2])
test['day'] = test['day'].astype(int)
print 'Step 2 Complete.'

Step 2 Complete.


In [7]:
# 3: Withdraw multiple date features
train['Date'] = pd.to_datetime(train['Date'])
train['day_of_year'] = train['Date'].dt.dayofyear
train['quarter'] = train['Date'].dt.quarter
train['is_month_start'] = train['Date'].dt.is_month_start.astype(int)
train['is_month_end'] = train['Date'].dt.is_month_end.astype(int)
train['is_quarter_start'] = train['Date'].dt.is_quarter_start.astype(int)
train['is_quarter_end'] = train['Date'].dt.is_quarter_end.astype(int)
print 'Step 3 Complete.'

Step 3 Complete.


In [8]:
lambdafunc = lambda x: pd.Series([x['Date'].isocalendar()[1]])
train[['weeknum']] = train.apply(lambdafunc, axis=1)

In [9]:
pd.isnull(train).any()

Store               False
DayOfWeek           False
Date                False
Sales               False
Customers           False
Open                False
Promo               False
StateHoliday        False
SchoolHoliday       False
year                False
month               False
day                 False
day_of_year         False
quarter             False
is_month_start      False
is_month_end        False
is_quarter_start    False
is_quarter_end      False
weeknum             False
dtype: bool

In [17]:
train[train.isnull().any(axis=1)]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,day_of_year,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,weeknum


In [11]:
# 3: Withdraw multiple date features
test['Date'] = pd.to_datetime(test['Date'])
test['day_of_year'] = test['Date'].dt.dayofyear
test['quarter'] = test['Date'].dt.quarter
test['is_month_start'] = test['Date'].dt.is_month_start.astype(int)
test['is_month_end'] = test['Date'].dt.is_month_end.astype(int)
test['is_quarter_start'] = test['Date'].dt.is_quarter_start.astype(int)
test['is_quarter_end'] = test['Date'].dt.is_quarter_end.astype(int)
print 'Step 3 Complete.'

Step 3 Complete.


In [12]:
# datetime.date(2015, 7, 31).isocalendar()[1]
lambdafunc = lambda x: pd.Series([x['Date'].isocalendar()[1]])
test[['weeknum']] = test.apply(lambdafunc, axis=1)

In [18]:
pd.isnull(test).any()

Id                  False
Store               False
DayOfWeek           False
Date                False
Open                False
Promo               False
StateHoliday        False
SchoolHoliday       False
year                False
month               False
day                 False
day_of_year         False
quarter             False
is_month_start      False
is_month_end        False
is_quarter_start    False
is_quarter_end      False
weeknum             False
dtype: bool

In [19]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,day_of_year,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,weeknum
0,1,1,4,2015-09-17,1,1,0,0,2015,9,17,260,3,0,0,0,0,38
1,2,3,4,2015-09-17,1,1,0,0,2015,9,17,260,3,0,0,0,0,38
2,3,7,4,2015-09-17,1,1,0,0,2015,9,17,260,3,0,0,0,0,38
3,4,8,4,2015-09-17,1,1,0,0,2015,9,17,260,3,0,0,0,0,38
4,5,9,4,2015-09-17,1,1,0,0,2015,9,17,260,3,0,0,0,0,38


In [20]:
# 4: Standardize StateHoliday column.
train.ix[(train['StateHoliday'] == 0), 'StateHoliday'] = '0'
print 'Step 4 Complete.'

Step 4 Complete.


In [21]:
# 4: Standardize StateHoliday column.
test.ix[(test['StateHoliday'] == 0), 'StateHoliday'] = '0'
print 'Step 4 Complete.'

Step 4 Complete.


In [22]:
# 5: Create duration of promo 1
train = train.sort(columns = ['Store', 'Date'], ascending = True, na_position = 'last')
train['PromoFirstDate'] = 0
train.ix[(train['Promo'] == 1 ) & (train['Promo'].shift(1) ==0), 'PromoFirstDate'] = 1
print 'Step 5 Complete.'

Step 5 Complete.


/Users/2015/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [23]:
# 5: Create duration of promo 1
test = test.sort(columns = ['Store', 'Date'], ascending = True, na_position = 'last')
test['PromoFirstDate'] = 0
test.ix[(test['Promo'] == 1 ) & (test['Promo'].shift(1) ==0), 'PromoFirstDate'] = 1
print 'Step 5 Complete.'

Step 5 Complete.


/Users/2015/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


# Join with States Data Frame

In [31]:
states.head()

,Store,State
0,1,HE
1,2,TH
2,3,NW
3,4,BE
4,5,SN


In [35]:
train.shape

(1017209, 20)

In [38]:
train_states = pd.merge(train, states, on = 'Store', how = 'inner')

In [39]:
train_states.shape

(1017209, 21)

In [40]:
train_states.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,...,day,day_of_year,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,weeknum,PromoFirstDate,State
0,1,2,2013-01-01,0,0,0,0,a,1,2013,...,1,1,1,1,0,1,0,1,0,HE
1,1,3,2013-01-02,5530,668,1,0,0,1,2013,...,2,2,1,0,0,0,0,1,0,HE
2,1,4,2013-01-03,4327,578,1,0,0,1,2013,...,3,3,1,0,0,0,0,1,0,HE
3,1,5,2013-01-04,4486,619,1,0,0,1,2013,...,4,4,1,0,0,0,0,1,0,HE
4,1,6,2013-01-05,4997,635,1,0,0,1,2013,...,5,5,1,0,0,0,0,1,0,HE


In [41]:
pd.isnull(train_states).any()

Store               False
DayOfWeek           False
Date                False
Sales               False
Customers           False
Open                False
Promo               False
StateHoliday        False
SchoolHoliday       False
year                False
month               False
day                 False
day_of_year         False
quarter             False
is_month_start      False
is_month_end        False
is_quarter_start    False
is_quarter_end      False
weeknum             False
PromoFirstDate      False
State               False
dtype: bool

In [42]:
def rstr(df): return df.shape, df.apply(lambda x: [x.unique()])
print(rstr(train_states))

((1017209, 21), Store               [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1...
DayOfWeek                                     [[2, 3, 4, 5, 6, 7, 1]]
Date                [[2013-01-01 00:00:00, 2013-01-02 00:00:00, 20...
Sales               [[0, 5530, 4327, 4486, 4997, 7176, 5580, 5471,...
Customers           [[0, 668, 578, 619, 635, 785, 654, 626, 615, 5...
Open                                                         [[0, 1]]
Promo                                                        [[0, 1]]
StateHoliday                                           [[a, 0, b, c]]
SchoolHoliday                                                [[1, 0]]
year                                             [[2013, 2014, 2015]]
month                       [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]
day                 [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1...
day_of_year         [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1...
quarter                                                [[1, 2, 3, 4]]
is_m

In [43]:
print(rstr(test))

((41088, 19), Id                  [[40233, 39377, 38521, 37665, 36809, 35953, 35...
Store               [[1, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1...
DayOfWeek                                     [[6, 7, 1, 2, 3, 4, 5]]
Date                [[2015-08-01 00:00:00, 2015-08-02 00:00:00, 20...
Open                                                     [[1.0, 0.0]]
Promo                                                        [[0, 1]]
StateHoliday                                                 [[0, a]]
SchoolHoliday                                                [[1, 0]]
year                                                         [[2015]]
month                                                        [[8, 9]]
day                 [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1...
day_of_year         [[213, 214, 215, 216, 217, 218, 219, 220, 221,...
quarter                                                         [[3]]
is_month_start                                               [[1, 0]]
is_mon

In [44]:
test.shape

(41088, 19)

In [45]:
test_states = pd.merge(test, states, on = 'Store', how = 'inner')
print(rstr(test_states))

((41088, 20), Id                  [[40233, 39377, 38521, 37665, 36809, 35953, 35...
Store               [[1, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1...
DayOfWeek                                     [[6, 7, 1, 2, 3, 4, 5]]
Date                [[2015-08-01 00:00:00, 2015-08-02 00:00:00, 20...
Open                                                     [[1.0, 0.0]]
Promo                                                        [[0, 1]]
StateHoliday                                                 [[0, a]]
SchoolHoliday                                                [[1, 0]]
year                                                         [[2015]]
month                                                        [[8, 9]]
day                 [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1...
day_of_year         [[213, 214, 215, 216, 217, 218, 219, 220, 221,...
quarter                                                         [[3]]
is_month_start                                               [[1, 0]]
is_mon

In [46]:
pd.isnull(test_states).any()

Id                  False
Store               False
DayOfWeek           False
Date                False
Open                False
Promo               False
StateHoliday        False
SchoolHoliday       False
year                False
month               False
day                 False
day_of_year         False
quarter             False
is_month_start      False
is_month_end        False
is_quarter_start    False
is_quarter_end      False
weeknum             False
PromoFirstDate      False
State               False
dtype: bool

In [47]:
test_states.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,day_of_year,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,weeknum,PromoFirstDate,State
0,40233,1,6,2015-08-01,1,0,0,1,2015,8,1,213,3,1,0,0,0,31,0,HE
1,39377,1,7,2015-08-02,0,0,0,1,2015,8,2,214,3,0,0,0,0,31,0,HE
2,38521,1,1,2015-08-03,1,1,0,1,2015,8,3,215,3,0,0,0,0,32,1,HE
3,37665,1,2,2015-08-04,1,1,0,1,2015,8,4,216,3,0,0,0,0,32,0,HE
4,36809,1,3,2015-08-05,1,1,0,1,2015,8,5,217,3,0,0,0,0,32,0,HE


# Output CSV files for R

In [48]:
pwd

u'/Users/2015/Dropbox/Data Science/NYC Data Science Academy/class-projects/Rossmann/Scripts/Python'

In [49]:
train_states.to_csv("../../KaggleProject/data/train_states_R.csv")

In [50]:
test_states.to_csv("../../KaggleProject/data/test_states_R.csv")

# Add mean sales features

In [ ]:
train_store.head(50)

In [ ]:
train.head(50)